<a href="https://colab.research.google.com/github/mattthelee/nlp-labs/blob/master/nnnlpLab10/Lab10_colabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 10: Dialogue Act Tagging

Dialogue act (DA) tagging is an important step in the process of developing dialog systems. DA tagging is a problem usually solved by supervised machine learning approaches that all require large amounts of hand labeled data. A wide range of techniques have been investigated for DA tagging. In this lab, we explore two models for DA classification. We are using the Switchboard Dialog Act Corpus for training.
Corpus can be downloaded from http://compprag.christopherpotts.net/swda.html.


The downloaded dataset should be kept in a data folder in the same directory as this file. 

In [1]:
import pandas as pd
import glob
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import pickle
import tensorflow as tf
from tqdm import tqdm


# tf.enable_eager_execution()


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

f = glob.glob("/content/gdrive/My Drive/swda/sw*/sw*.csv")
frames = []
for i in range(0, len(f)):
    frames.append(pd.read_csv(f[i]))

result = pd.concat(frames, ignore_index=True)




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
print("Number of converations in the dataset:",len(result))


Number of converations in the dataset: 223606


The dataset has many different features, we are only using act_tag and text for this training.


In [0]:
reduced_df = result[['act_tag','text']]


In [5]:
reduced_df.head()


,act_tag,text
0,qo,"Um, so what do you think? /"
1,^h,"Um, -/"
2,x,<Baby_talking>.
3,sd,"I lived, I lived in Sweden for two years. /"
4,b,Uh-huh. /


Theere are 43 tags in this dataset. Some of the tags are Yes-No-Question('qy'), Statement-non-opinion('sd') and Statement-opinion('sv'). Tags information can be found here http://compprag.christopherpotts.net/swda.html#tags. 


You can check the frequency of tags.

In [6]:
act_tag_counts = reduced_df['act_tag'].value_counts()
# Reduce the number of classes by replacing rare labels with the 'other' token
# By reducing the number i help to reduce the class imbalance 
# and also scale the confusion matrix down to a size that allows for interpretation
threshold = 100
for label in act_tag_counts.keys():
    if act_tag_counts[label] < threshold:
        reduced_df.replace(label,'other', inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


## Baseline BiLSTM Model

To get unique tags. 

In [0]:
unique_tags = set()
for tag in reduced_df['act_tag']:
    unique_tags.add(tag)

In [0]:
# Load the onehot encoding dataframe from disk to avoid re-runs incorrectly mapping vectors
import os.path
fname = "one_hot_encoding_dic.pickle"
if os.path.isfile(fname) and False:
    pickle_in = open(fname,"rb")
    one_hot_encoding_dic = pickle.load(pickle_in)
else:
    one_hot_encoding_dic = pd.get_dummies(list(unique_tags))


pickle_out = open(fname,"wb")
pickle.dump(one_hot_encoding_dic, pickle_out)
pickle_out.close()

In [0]:
tags_encoding = []
for i in range(0, len(reduced_df)):
    tags_encoding.append(one_hot_encoding_dic[reduced_df['act_tag'].iloc[i]])

The tags are one hot encoded.

To create sentence embeddings:

In [0]:
sentences = []
for i in range(0, len(reduced_df)):
    sentences.append(reduced_df['text'].iloc[i].split(" "))


In [0]:
wordvectors = {}
index = 1
for s in sentences:
    for w in s:
        if w not in wordvectors:
            wordvectors[w] = index
            index += 1

In [12]:
MAX_LENGTH = len(max(sentences, key=len))
#MAX_LENGTH = 150
# needs to be an even number of tensorflow reshaping to work
if MAX_LENGTH % 2 == 1:
    MAX_LENGTH += 1
print(MAX_LENGTH)

138


In [0]:
sentence_embeddings = []
for s in sentences:
    sentence_emb = []
    for w in s:
        sentence_emb.append(wordvectors[w])
    sentence_embeddings.append(sentence_emb)


Then we split the dataset into test and train.

In [0]:
from sklearn.model_selection import train_test_split
import numpy as np
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings, np.array(tags_encoding))


And pad the sentences with zero to make all sentences of equal length.


In [0]:
from keras.preprocessing.sequence import pad_sequences
 
train_sentences_X = pad_sequences(X_train, maxlen=MAX_LENGTH, padding='post')
test_sentences_X = pad_sequences(X_test, maxlen=MAX_LENGTH, padding='post')

 The model architecture is as follows: Embedding Layer (to generate word embeddings) Next layer Bidirectional LSTM. Feed forward layer with number of neurons = number of tags. Softmax activation to get probabilities.

In [16]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout, InputLayer, Bidirectional, TimeDistributed, Activation, Embedding
from keras.optimizers import Adam
#Building the network
numberOfTags = len(reduced_df['act_tag'].value_counts())
embed_dim = 50

model = Sequential()
model.add(Embedding(len(wordvectors)+1, output_dim=embed_dim, input_length=MAX_LENGTH))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(numberOfTags,activation='softmax' ))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
model.summary()
weightsFilePath ="weights.best.hdf5"

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 138, 50)           2186600   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               58880     
_________________________________________________________________
dense_1 (Dense)              (None, 58)                7482      
Total params: 2,252,962
Trainable params: 2,252,962
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Save the best weights to a file so we get the model with the best val acc

'''
# Commented out to allow working on other things without running this cell
checkpoint = ModelCheckpoint(weightsFilePath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model.fit(train_sentences_X,y_train,epochs=5,batch_size=512,validation_data=(test_sentences_X, y_test), callbacks=[checkpoint],verbose=1)
'''

"\n# Commented out to allow working on other things without running this cell\ncheckpoint = ModelCheckpoint(weightsFilePath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')\nhistory = model.fit(train_sentences_X,y_train,epochs=5,batch_size=512,validation_data=(test_sentences_X, y_test), callbacks=[checkpoint],verbose=1)\n"

In [0]:
#model.load_weights(weightsFilePath)
#score = model.evaluate(test_sentences_X, y_test, batch_size=100)

In [19]:
bfTotal = 0.0
brTotal = 0.0
sdTotal = 0.0
for i,row in enumerate(y_test):
    if np.array_equal(row,one_hot_encoding_dic['br']):
        brTotal += 1.0
    elif np.array_equal(row,one_hot_encoding_dic['bf']):
        bfTotal += 1.0
    elif np.array_equal(row,one_hot_encoding_dic['sd']):
        sdTotal += 1.0
        
print(bfTotal)
print(brTotal)
print(sdTotal)

239.0
47.0
17577.0


In [20]:
for i,key in enumerate(one_hot_encoding_dic.keys()):
    if key == 'br':
        brConfusionMatrixIndex = i
    elif key == 'bf':
        bfConfusionMatrixIndex = i
    elif key == 'sd':
        sdConfusionMatrixIndex = i
print(brConfusionMatrixIndex)
print(bfConfusionMatrixIndex)
print(sdConfusionMatrixIndex)

19
16
45


In [0]:
# Commented out so I can run notebook without taking time to do this 
# y_pred = model.predict(test_sentences_X)

In [22]:
# commented out for running efficieny
'''

# Create dict to convert y vectors back to strings
vec_to_string = {}
for key in one_hot_encoding_dic.keys():
    vec_to_string[one_hot_encoding_dic[key].argmax()] = key

y_pred_strings = []
for vec in y_pred:
    y_pred_strings.append(vec_to_string[vec.argmax()])

y_test_strings = []
for vec in y_test:
    y_test_strings.append(vec_to_string[vec.argmax()])
    
cm = confusion_matrix(y_test_strings,y_pred_strings, labels=one_hot_encoding_dic.keys())
brAccuracy = float(cm[brConfusionMatrixIndex][brConfusionMatrixIndex] )/ float(brTotal)
bfAccuracy = float(cm[bfConfusionMatrixIndex][bfConfusionMatrixIndex]) / float(bfTotal)
sdAccuracy = float(cm[sdConfusionMatrixIndex][sdConfusionMatrixIndex]) / float(sdTotal)
print(f"br Accuracy: {brAccuracy}, bf accuracy: {bfAccuracy},sd Accuracy: {sdAccuracy} ")
print(f"Overall accuracy: {np.diag(cm).sum()*100.0 / len(y_test)}")

brCommonMistakes = cm[brConfusionMatrixIndex]
print(f"The br was most commonly confused with: {one_hot_encoding_dic.keys()[brCommonMistakes.argmax()]}")

bfCommonMistakes = cm[bfConfusionMatrixIndex]
print(f"The bf was most commonly confused with: {one_hot_encoding_dic.keys()[bfCommonMistakes.argmax()]}")

# The br and bf sentences are incorrectly labelled as aa and sd. 
# I think this is because aa and sd are the most common classes in the dataset
# The 0% accuracy for bf and br was concering so i decided to plot thw whole confusion matrix to see what was happening
# The confusion matrix is mostly dark with a small number of bright dots. These are the classes it tries to classify
# This indicates that the large class imbalance has caused the model to only output a small number of classes rather than the full range
# It simply accepts it will get the other classes wrong as it is still able to achieve good (>60%) accuracy without using them
plt.imshow(cm)
plt.colorbar()

plt.show()

plt.savefig("confusionmatrix.png")
'''

'\n\n# Create dict to convert y vectors back to strings\nvec_to_string = {}\nfor key in one_hot_encoding_dic.keys():\n    vec_to_string[one_hot_encoding_dic[key].argmax()] = key\n\ny_pred_strings = []\nfor vec in y_pred:\n    y_pred_strings.append(vec_to_string[vec.argmax()])\n\ny_test_strings = []\nfor vec in y_test:\n    y_test_strings.append(vec_to_string[vec.argmax()])\n    \ncm = confusion_matrix(y_test_strings,y_pred_strings, labels=one_hot_encoding_dic.keys())\nbrAccuracy = float(cm[brConfusionMatrixIndex][brConfusionMatrixIndex] )/ float(brTotal)\nbfAccuracy = float(cm[bfConfusionMatrixIndex][bfConfusionMatrixIndex]) / float(bfTotal)\nsdAccuracy = float(cm[sdConfusionMatrixIndex][sdConfusionMatrixIndex]) / float(sdTotal)\nprint(f"br Accuracy: {brAccuracy}, bf accuracy: {bfAccuracy},sd Accuracy: {sdAccuracy} ")\nprint(f"Overall accuracy: {np.diag(cm).sum()*100.0 / len(y_test)}")\n\nbrCommonMistakes = cm[brConfusionMatrixIndex]\nprint(f"The br was most commonly confused with: {on

In addition to overall accuracy, you need to look at the accuracy of some minority classes. Signal-non-understanding ('br') is a good indicator of "other-repair" or cases in which the other conversational participant attempts to repair the speaker's error. Summarize/reformulate ('bf') has been used in dialogue summarization. Report the accuracy for these classes and some frequent errors you notice the system makes in predicting them. What do you think the reasons are？


As the dataset is highly imbalanced, we can simply weight up the minority classes proportionally to their underrepresentation while training. 

In [0]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(tags_encoding, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
classBalancedWeightsFilePath = "classBalanced" + weightsFilePath

In [0]:

classBalancedCheckpoint = ModelCheckpoint(classBalancedWeightsFilePath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
# Commented out to allow running of whole notebook efficiently
#model.fit(train_sentences_X, y_train, batch_size=512, epochs=5, class_weight = d_class_weights, validation_data=(test_sentences_X, y_test), callbacks=[classBalancedCheckpoint])

Report the overall accuracy and the accuracy of  'br' and 'bf'  classes. Suggest other ways to handle imbalanced classes.

Can we improve things by using context information?  Next we try to build a model which predicts DA tag from the sequence of 
previous DA tags, plus the utterance representation. 

##Using Context for Dialog Act Classification
We expect there is valuable sequential information among the DA tags. So in this section we apply a BiLSTM on top of the sentence CNN representation. The CNN model learns textual information in each sentence for DA classification. Here, we use bidirectional-LSTM (BLSTM) to learn the context before and after the current sentence. The left-to-right LSTM output and the one from the reverse direction are concatenated and input to a hidden layer for classification.

Functions for creating weights and biases.

In [0]:
def weights_init(shape):
    return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.05))

In [0]:
def bias_init(shape):
    return tf.Variable(tf.zeros(shape=shape))


 This is classical CNN layer used to convolve over embedings tensor and gether useful information from it. The data is represented by hierarchy of features, which can be modelled using a CNN.
    
      Input(s): 
              input - word_embedings
              filter_size - size of width and height of the Conv kernel
              number_of_channels - in this case it is always 1
              number_of_filters - how many representation of the input utterance are we going to output from this layer 
              strides - how many does kernel move to the side and up/down
              activation - a activation function
              max_pool - boolean value which will trigger a max_pool operation on the output tensor
      Output(s): 
               text_conv layer
    

In [0]:
def text_conv(input, filter_size, number_of_channels, number_of_filters, strides=(1, 1), activation=tf.nn.relu, max_pool=True):
    #print('Conv')
    #print(input)
    #print(filter_size)
    #print('conv end')
    weights = weights_init([filter_size, filter_size, number_of_channels, number_of_filters])
    print(tf.shape(input))
    bias = bias_init([number_of_filters])
    print('bias done')
    layer = tf.nn.conv2d(input, filter=weights, strides=[1, strides[0], strides[1], 1], padding='SAME')
    print('done')
    if activation != None:
        layer = activation(layer)
    print('this works')
    if max_pool:
        layer = tf.nn.max_pool(layer, ksize=[1, 2, 2 ,1], strides=[1, 2, 2, 1], padding='SAME')
    
    return layer

    This method is used to create LSTM layer. And the data we’re working with has temporal properties which we want to model as well — hence the use of a LSTM. You can create a BiLSTM by modifying this.
    
    Input(s): lstm_cell_unitis - used to define the number of units in a LSTM layer
              number_of_layers - used to define how many of LSTM layers do we want in the network
              batch_size - in this method this information is used to build starting state for the network
              dropout_rate - used to define how many cells in a layer do we want to 'turn off'
              
    Output(s): cell - lstm layer
               init_state - zero vectors used as a starting state for the network

In [0]:
def lstm_layer(lstm_size, number_of_layers, batch_size, dropout_rate):

    def cell(size, dropout_rate=None):
        layer = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        return tf.contrib.rnn.DropoutWrapper(layer, output_keep_prob=dropout_rate)
            
    cell = tf.contrib.rnn.MultiRNNCell([cell(lstm_size, dropout_rate) for _ in range(number_of_layers)])
    
    init_state = cell.zero_state(batch_size, tf.float32)
    return cell, init_state

    Use to transform/reshape conv output to 2d matrix, if it's necessary
    
    Input(s): Layer - text_cnn layer
              batch_size - how many samples do we feed at once
              seq_len - number of time steps
              
    Output(s): reshaped_layer - the layer with new shape
               number_of_elements - this param is used as a in_size for next layer

In [0]:
def flatten(layer, batch_size, seq_len):

    dims = layer.get_shape()
    number_of_elements = dims[2:].num_elements()
    
    reshaped_layer = tf.reshape(layer, [batch_size, int(seq_len/2), number_of_elements])
    return reshaped_layer, number_of_elements

    Output layer for the lstm netowrk
    
    Input(s): lstm_outputs - outputs from the RNN part of the network
              input_size - in this case it is RNN size (number of neuros in RNN layer)
              output_size - number of neuros for the output layer == number of classes
              
    Output(s) - logits, 

In [0]:
def dense_layer(input, in_size, out_size, dropout=False, activation=tf.nn.relu):
  
    weights = weights_init([in_size, out_size])
    bias = bias_init([out_size])
    
    layer = tf.matmul(input, weights) + bias
    
    if activation != None:
        layer = activation(layer)
    
    if dropout:
        layer = tf.nn.dropout(layer, 0.5)
        
    return layer

    Function used to calculate loss and minimize it
    
    Input(s): rnn_out - logits from the fully_connected layer
              targets - targets used to train network
              learning_rate/step_size
    
    
    Output(s): optimizer - optimizer of choice
               loss - calculated loss function

In [0]:
def loss_optimizer(logits, targets, learning_rate ):
    loss =tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=targets))
    print(loss)
    print(targets)
    print(logits)

    print(learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return loss, optimizer

To create the model you can use these inputs:     
       
       Input(s): learning_rate/step_size - how fast are we going to find global minima
                  batch_size -  the nuber of samples to feed at once
                  seq_len - the number of timesteps in unrolled RNN
                  vocab_size - the number of nunique words in the vocab
                  embed_size - length of word embed vectors
                  conv_filters - number of filters in output tensor from CNN layer
                  conv_filter_size - height and width of conv kernel
                  number_of_lstm_layers - the number of layers used in the LSTM part of the network
                  lstm_units - the number of neurons/cells in a LSTM layer

In [0]:
class DATagging(object):
    
    def __init__(self, learning_rate=0.001, batch_size=100, seq_len=250, vocab_size=10000, embed_size=300,
                conv_filters=32, conv_filter_size=5, number_of_lstm_layers=1, lstm_units=128):
        
        # Clear tf graph for multiple cell running
        tf.reset_default_graph()
        
        # Create placeholders
        self.inputSentences = tf.placeholder(tf.int32, [batch_size, seq_len], name='inputSentences')
        self.targetLabels= tf.placeholder(tf.float32, [batch_size, numberOfTags], name='targetLabels')    

        
        
        #Embedding layer
        # Initialise word embeddings with random uniform dist
        wordEmbeddings = tf.Variable(tf.random_uniform([vocab_size, embed_size]))
        # Embedding layer
        embeddingLayer = tf.nn.embedding_lookup(wordEmbeddings, self.inputSentences)
        
        # Reshape embedding layer to have a "channel"
        embeddingLayer = tf.expand_dims(embeddingLayer, -1)

        # Conv layer
        number_of_channels = 1
        sentence_embedding = text_conv(embeddingLayer, conv_filter_size,number_of_channels, conv_filters)
        flat_sentence_embedding, size = flatten(sentence_embedding, batch_size, seq_len)
        
        
        lstm_cell, init_state = lstm_layer(lstm_units, number_of_lstm_layers, batch_size, dropout_rate = 0.3)

        # single LSTM
        outputs,states = tf.nn.dynamic_rnn(lstm_cell, flat_sentence_embedding,
                                                       dtype=tf.float32)
        shapedOutputs = tf.squeeze(tf.slice(outputs,[0,0,0],[-1,1,-1]))
        denseLayer = dense_layer(shapedOutputs, lstm_units, numberOfTags, activation=None)

        '''
        # BiLSTM layers
        
        (outputs,states) = tf.nn.bidirectional_dynamic_rnn(lstm_cell,
                                                       lstm_cell, 
                                                       flat_sentence_embedding,
                                                       dtype=tf.float32)
        
        
        # Need to reshape the outputs to be include only the last output from the sequence, 
        # but every instance in batch and every lstm output
        (forwardOutputs,backwardOutputs) = outputs
        print('***')
        print(forwardOutputs)
        print(tf.squeeze(tf.slice(forwardOutputs, [0,-1,0],[-1,1,-1])))
        reshapedForward = tf.squeeze(tf.slice(forwardOutputs, [0,-1,0],[-1,1,-1]))
        reshapedBackward = tf.squeeze(tf.slice(backwardOutputs, [0,-1,0],[-1,1,-1]))
        shapedOutputs = tf.concat([reshapedForward,reshapedBackward], 1)
        #shapedOutputs = tf.concat(outputs,2)
        
        print(shapedOutputs)
        # Hidden layer, with relu activation as we want it to output word embeddings
        denseLayer = dense_layer(shapedOutputs, lstm_units*2, numberOfTags)
        '''
        #print(denseLayer)
        #print(self.targetLabels)
        self.loss, self.optimizer = loss_optimizer(denseLayer, self.targetLabels, learning_rate)
        
        predictions = tf.math.argmax(denseLayer, axis = 1)
        labels = tf.argmax(self.targetLabels, axis = 1)
        #print(predictions)
        #print(labels)
        
        self.accuracy = tf.metrics.accuracy(labels, predictions)[1]

In [0]:
def shuffleData(x_train,y_train):
    # Returns a shuffled batch of x and y data
    indices = np.random.permutation(len(x_train))
    x_train_shuffled = x_train[indices]
    y_train_shuffled = y_train[indices]
    return x_train_shuffled, y_train_shuffled


In [37]:
batch_size = 256
model2 = DATagging(learning_rate=0.001, 
                     batch_size=batch_size, 
                     seq_len=MAX_LENGTH, 
                     vocab_size=len(wordvectors) + 1, 
                     embed_size=50,
                     conv_filters=32, 
                     conv_filter_size=5, 
                     number_of_lstm_layers=1, 
                     lstm_units=128)

session = tf.Session()
session.run(tf.global_variables_initializer())
session.run(tf.local_variables_initializer())
epochs = 5
for i in range(epochs):
    epoch_loss = []
    train_accuracy = []
    for ii in tqdm(range(0, len(train_sentences_X), batch_size)):
        x_train_shuffled, y_train_shuffled = shuffleData(train_sentences_X,y_train)
        X_batch = x_train_shuffled[ii:ii+batch_size]
        y_batch = y_train_shuffled[ii:ii+batch_size]
        if len(y_batch) < batch_size:
            # If we are at the end of the data, take  batch from teh end, 
            # this repeats some data but allows us to use all of it
            X_batch = x_train_shuffled[-batch_size:]
            y_batch = y_train_shuffled[-batch_size:]
        
        c, _, a = session.run([model2.loss, model2.optimizer, model2.accuracy], feed_dict={model2.inputSentences:X_batch, 
                                                                                  model2.targetLabels:y_batch})
        print(f"Accuracy: {a}")
        epoch_loss.append(c)
        train_accuracy.append(a)
        
    
    print("Epoch: {}/{}".format(i, epochs), " | Epoch loss: {}".format(np.mean(epoch_loss)), 
          " | Mean train accuracy: {}".format(np.mean(train_accuracy)))
#model2.summary()
#history = model2.fit(train_sentences_X,y_train, epochs=5)

Tensor("Shape:0", shape=(4,), dtype=int32)
bias done
done
this works
Tensor("Mean:0", shape=(), dtype=float32)
Tensor("targetLabels:0", shape=(256, 58), dtype=float32)
Tensor("add:0", shape=(256, 58), dtype=float32)
0.001




  0%|          | 0/656 [00:00<?, ?it/s]

  0%|          | 1/656 [00:00<05:03,  2.16it/s]

Accuracy: 0.01953125




  0%|          | 2/656 [00:00<04:27,  2.44it/s]

Accuracy: 0.134765625


...

Accuracy: 0.5665706992149353




100%|█████████▉| 655/656 [02:46<00:00,  3.90it/s]

Accuracy: 0.5665849447250366




100%|██████████| 656/656 [02:46<00:00,  3.90it/s]



Accuracy: 0.5665920376777649
Epoch: 4/5  | Epoch loss: 1.4475971460342407  | Mean train accuracy: 0.5617845058441162


Compared to the baseline using BiLSTM for utterance classification, the second method effectively leverage context information and achieve better performance. Report your overall accuracy. Did context help disambiguate and better predict the minority classes ('br' and 'bf')? What are frequent errors? Show one positive example where adding context changed the prediction.




## Advanced: Creating End-To-End Dialogue System

In the last section we want to create end-to-end dialogue system, following on from the seq2seq MT labs you've 
just done. This is an advanced part of the assignment and worth 10 marks (20%) in total. In end-to-end dialogue systems, the encoder represents each utterance with a vector. The utterance vector is the hidden state after the last token of the utterance has been processed. The context LSTM keeps track of past utterances. The hidden state can be explained as the state of the dialogue system. The next utterance prediction is performed by a decoder LSTM, which takes the hidden state of the last LSTM and produces a probability distribution over the tokens in the next utterance. You can take the DA LSTM state of last section as input to a decoder which tries to generate the next utterance. You can add attention and monitor the performance. Instead of evaluating by an automatic evaluation method, you can show us some of the interesting predictions. 
